In [1]:
import datetime
import numpy as np
import os
from pathos import multiprocessing as mp

In [2]:
data_path = "/home/trevor/Downloads/Toroidal_topology_grid_cell_data/rat_r_day1_grid_modules_1_2_3.npz"

In [3]:
with np.load(data_path, allow_pickle=True) as data:
    module_1_spikes = data['spikes_mod1'][()]
    module_2_spikes = data['spikes_mod2'][()]
    module_3_spikes = data['spikes_mod3'][()]
    x_pos = data['x']
    y_pos = data['y']
    z_pos = data['z']
    times = data['t']

In [4]:
time_range = [np.min(times), np.max(times)]
time_grid = np.linspace(start=np.min(times), stop=np.max(times), num=int(time_range[1] - time_range[0]))

In [5]:
def sum_of_gaussians(times, centers, sigma):
    activation = np.zeros_like(times)
    for i, center in enumerate(centers):
        activation += np.exp(-(times - center)**2/(2 * sigma**2))
    return activation

In [6]:
all_rate_data = []
for module_spikes in [module_1_spikes, module_2_spikes, module_3_spikes]:
    all_spike_times = []
    for key, value in module_spikes.items():
        all_spike_times.append(value)

    p = mp.Pool()
    rate_data = p.map(lambda d: sum_of_gaussians(time_grid, d, 0.1), all_spike_times)
    rate_data = np.array(rate_data)
    all_rate_data.append(rate_data)
all_rate_data = np.array(all_rate_data)

/tmp/ipykernel_241494/257980028.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_rate_data = np.array(all_rate_data)


In [12]:
output_dir = "/home/trevor/manifold_test_data/torus/gardner_data/{}".format(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
os.makedirs(output_dir, exist_ok=False)

In [13]:
np.save(os.path.join(output_dir, "all_mod_encoded_points.npy"), all_rate_data)

In [14]:
positions = np.stack([x_pos, y_pos, z_pos], axis=-1)

In [15]:
np.save(os.path.join(output_dir, "positions.npy"), positions)
np.save(os.path.join(output_dir, "times.npy"), times)


In [16]:
np.save(os.path.join(output_dir, "encoded_point_time_bins.npy"), time_grid)
